In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ベースURL
base_url = "https://www.jalan.net/ikisaki/map/tokyo/"
response = requests.get(base_url)

# メタタグからエンコーディングを取得して設定
soup = BeautifulSoup(response.content, "html.parser")
meta_tag = soup.find("meta", {"charset": True})

if meta_tag and meta_tag.get("charset"):
    response.encoding = meta_tag.get("charset")
else:
    response.encoding = response.apparent_encoding

# 再度パース
soup = BeautifulSoup(response.text, "html.parser")

# 'map-right-inner' を持つ div を取得
map_right_inner_div = soup.find("div", class_="map-right-inner")
if map_right_inner_div:
    a_tags = map_right_inner_div.find_all("a", class_="sub")
    print(f"発見した a タグの数: {len(a_tags)}")
else:
    print("対象の div が見つかりませんでした。")
    a_tags = []

# URLを収集
urls = [urljoin(base_url, a['href']) for a in a_tags if 'href' in a.attrs]

# 特定の文字化けを修正する関数
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # 修正できない場合そのまま返す

# 各URLにアクセスして詳細情報を取得
for i, url in enumerate(urls, 1):
    print(f"{i}: {url}")
    response = requests.get(url)

    # メタタグでエンコーディングを確認して設定
    detail_soup = BeautifulSoup(response.content, "html.parser")
    meta_tag = detail_soup.find("meta", {"charset": True})
    if meta_tag and meta_tag.get("charset"):
        response.encoding = meta_tag.get("charset")
    else:
        response.encoding = response.apparent_encoding

    # 再度HTMLをパース
    detail_soup = BeautifulSoup(response.text, "html.parser")

    # ターゲットのデータを収集
    div_body = detail_soup.find("div", class_="p-yadoCassette__body p-searchResultItem__body")
    if div_body:
        # ホテル名
        h2_tag = div_body.find("h2", class_="p-searchResultItem__facilityName")
        raw_hotel_name = h2_tag.get_text(strip=True) if h2_tag else "ホテル名なし"
        hotel_name = fix_encoding(raw_hotel_name)

        # 詳細ページURL
        a_tag = div_body.find("a", class_="jlnpc-yadoCassette__link s16_00 fb")
        detail_url = a_tag['data-href'] if a_tag and 'data-href' in a_tag.attrs else "URLなし"

        # アクセス情報
        access_tag = div_body.find("dd", class_="p-searchResultItem__accessValue")
        access_info = access_tag.get_text(strip=True) if access_tag else "アクセス情報なし"

        # 価格情報
        price_tag = div_body.find("span", class_="p-searchResultItem__lowestPriceValue")
        price_info = price_tag.get_text(strip=True) if price_tag else "価格情報なし"

        # 一人あたり価格
        unit_price_tag = div_body.find("span", class_="p-searchResultItem__lowestUnitPrice")
        unit_price_info = unit_price_tag.get_text(strip=True) if unit_price_tag else "一人あたり価格情報なし"

        # ホテルの評価
        rating_tag = div_body.find("span", class_="p-searchResultItem__summaryaverage-num")
        rating_info = rating_tag.get_text(strip=True) if rating_tag else "評価なし"

        # レビュー数
        review_count_tag = div_body.find("span", class_="score-number")
        review_count_info = review_count_tag.get_text(strip=True) if review_count_tag else "レビュー数なし"

        # 情報を表示
        print(f"ホテル名: {hotel_name}")
        print(f"詳細ページURL: {detail_url}")
        print(f"アクセス情報: {access_info}")
        print(f"価格情報: {price_info}")
        print(f"一人あたり価格: {unit_price_info}")
        print(f"ホテルの評価: {rating_info}")
        print(f"レビュー数: {review_count_info}")
    else:
        print("対象の情報が見つかりませんでした。")
    print("-" * 50)


発見した a タグの数: 13
1: https://www.jalan.net/130000/LRG_137100/
ホテル名: 儂僥儖僀儞僞乕僐儞僠僱儞僞儖搶嫗儀僀
詳細ページURL: javascript:doYadDetailAd('318359','TGS01419110','115_1_1','2');
アクセス情報: 昹徏挰墂搆曕8暘丂戝峕屗慄/愺憪慄戝栧墂搆曕10暘丂備傝偐傕傔抾幣墂捈寢
価格情報: 50,900墌乣
一人あたり価格: 1柤 25,450墌乣
ホテルの評価: 4.8
レビュー数: 4.8
--------------------------------------------------
2: https://www.jalan.net/130000/LRG_138000/
ホテル名: 吉祥寺東急REIホテル
詳細ページURL: javascript:doYadDetailAd('311450','TGS01419146','137_1_1','2');
アクセス情報: ＪＲ中央線・京王井の頭線「吉祥寺駅」公園口（南口）より徒歩１分！
価格情報: 14,000円〜
一人あたり価格: 1名 7,000円〜
ホテルの評価: 4.2
レビュー数: 4.2
--------------------------------------------------
3: https://www.jalan.net/130000/LRG_136200/
ホテル名: TSUKI 搶嫗
詳細ページURL: javascript:doYadDetailAd('388628','TGS01419074','209_1_1','2');
アクセス情報: 搶嫗儊僩儘擔斾扟慄乽抸抧墂乿傛傝搆曕栺3暘 搶嫗儊僩儘桳妝挰慄乽怴晉挰墂乿傛傝搆曕栺4暘
価格情報: 23,200墌乣
一人あたり価格: 1柤 11,600墌乣
ホテルの評価: 評価なし
レビュー数: レビュー数なし
--------------------------------------------------
4: https://www.jalan.net/130000/LRG_137400/
ホテル名: アゴーラプレイス 東京浅草
詳細ページURL: javascript:doYadDetailAd(